In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import PIL
from PIL import Image
import random

import csv

/Users/noon/src/python/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#メソッドまとめ
#  1.  augument_image(output_dirs, orig_dirs)
#  2.  resize_images(target_dirs, base_dirs, data_names, data_nums, new_size)
#  3.  make_data(dataset_index)
#  4.  generate_test_train_data(data, label, test_size)
#  5.  save_test_train_data(data, label, test_size, dataset_index=-1)

In [3]:
def augument_image(output_dirs, orig_dirs):
    #引数
    # ouput_dir: 出力先のパス
    # orig_dir: 元データのパス
    
    for output_dir, orig_dir in zip(output_dirs, orig_dirs):
        if not(os.path.exists(output_dir)):
            os.mkdir(output_dir)

        # 拡張する画像群の読み込み
        images = glob.glob(os.path.join(orig_dir+'/', "*.jpeg"))

        # 拡張する際の設定
        generator = ImageDataGenerator(
                        rotation_range=90, # 90°まで回転
                        width_shift_range=0.1, # 水平方向にランダムでシフト
                        height_shift_range=0.1, # 垂直方向にランダムでシフト
                        channel_shift_range=0.0, # 色調をランダム変更
                        shear_range=0.39, # 斜め方向(pi/8まで)に引っ張る
                        horizontal_flip=True, # 垂直方向にランダムで反転
                        vertical_flip=True # 水平方向にランダムで反転
                        )

        # 読み込んだ画像を順に拡張
        for i in range(len(images)):
            img = load_img(images[i])
            # 画像を配列化して転置a
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            # 画像の拡張
            draw_images(generator, x, output_dir, i)

In [4]:
# 画像の枚数
# mukudori: 1139
# suzume: 970

def resize_images(target_dirs, base_dirs, data_names, data_nums, new_size):
    #引数
    # target_dirs: 出力先のパス
    # base_dirs: 元データのパス
    # data_nums: 元データのデータ枚数
    # new_size:　リサイズ先の一辺の長さ(もともとは28x28だった)
    
    for target_dir, base_dir, data_name, data_num in zip(target_dirs, base_dirs, data_names, data_nums):
        for i in range(data_num):
            index = str(i)
            image = Image.open(base_dir+dir_name+"/"+dir_name+"_"+index+".jpeg")
            image = image.resize((new_size,new_size))

            if(os.path.isfile(target_dir+dir_name+"/"+dir_name+"_"+index+".jpeg")):
                os.remove(target_dir+dir_name+"/"+dir_name+"_"+index+".jpeg")
            image.save(target_dir+dir_name+"/"+dir_name+"_"+index+".jpeg")

In [5]:
def make_data(dataset_index):
    data_names = ["mukudori", "suzume"]
    data_nums = [1139, 968] #[m_num, s_num]
    datas = [[], []] #[m_label, s_label]
    labels = [[], []]
    data_dirs = ['./dataset_'+dataset_index+'/resized/mukudori/', './dataset_'+dataset_index+'/resized/suzume/'] #m_dir, s_dir

    for data_num, data, label, data_dir, data_name in zip(data_nums, datas, labels, data_dirs, data_names):
        for i in range(data_num):
            data.append(0)
            if data_name == 'mukudori':
                label.append([1, 0])
            else:
                label.append([0, 1])
        
        for i in range(data_num):
            index = str(i)
            data[i] = Image.open(data_dir+data_name+'_'+index+'.jpeg')
            width, height = data[i].size
            
            img_pixels = []
            for y in range(height):
                for x in range(width):
                    img_pixels.append(data[i].getpixel((x, y)))

            data[i] = np.array(img_pixels)
            data[i] = np.reshape(data[i], (28, 28, 3))
    
    concat_data = datas[0] + datas[1]
    concat_label = labels[0] + labels[1]
    
    return concat_data, concat_label

In [6]:
def generate_test_train_data(data, label, test_size):
    return  train_test_split(data, label, test_size=0.3)
    # Data_train, Data_test, Label_train, Label_test の形式で帰ってくる

In [7]:
def save_test_train_data(data, label, test_size, dataset_index=-1):
    D_train, D_test, L_train, L_test = generate_test_train_data()
    
    if not data_set_index == -1:
        index = str(dataset_index)
        np.save('./dataset_'+index+'/data/D_train', D_train)
        np.save('./dataset_'+index+'/data/D_test', D_test)
        np.save('./dataset_'+index+'/data/L_train', L_train)
        np.save('./dataset_'+index+'/data/L_test', L_test)

In [54]:
NUM_CLASSES = 2
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE * 3

flags = tf.app.flags
FLAGS = flags.FLAGS
# flags.DEFINE_string('image_dir', 'dataset_1/resized', 'Directory of images')
# flags.DEFINE_string('train_dir', 'logs', 'Directory to put the training data.')
# flags.DEFINE_integer('max_steps', 200, 'Number of steps to run trainer.')
# flags.DEFINE_integer('batch_size', 10, 'Batch size'
#                      'Must divide evenly into the dataset sizes.')
# flags.DEFINE_float('learning_rate', 1e-5, 'Initial learning rate.')
image_dir = 'dataset_2/images'
train_dir = 'dataset_2/logs'
max_steps = 200
batch_size = 10
learning_rate = 1e-5

def inference(images_placeholder, keep_prob):
    #重みをstd=0.1で初期化
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    #バイアスをstd=0.1で初期化
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    #畳み込み層の作成
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
    
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    x_image = tf.reshape(images_placeholder, [-1, 28, 28, 3])
    
    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5, 5, 3, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
        tf.summary.histogram("wc1", W_conv1)
    
    with tf.name_scope('pool1') as scope:
        h_pool1 = max_pool_2x2(h_conv1)
    
    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        tf.summary.histogram("wc2", W_conv2)
    
    with tf.name_scope('pool2') as scope:
        h_pool2 = max_pool_2x2(h_conv2)
    
    with tf.name_scope('fc1') as scope:
        W_fc1 = weight_variable([7*7*64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    with tf.name_scope('fc2') as scope:
        W_fc2 = weight_variable([1024, NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])
    
    with tf.name_scope('softmax') as scope:
        y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    
    return y_conv

def loss(logits, labels):
    #交差エントロピーの計算
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits))
    #tensorboard用
    tf.summary.scalar("cross_entropy", cross_entropy)
    return cross_entropy

def training(loss, learning_rate):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step

def accuracy(logits, labels):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar("accuracy", accuracy)
    return accuracy

In [70]:
# data, label = make_data(str(1))
# train_image, test_image, train_label, test_label = generate_test_train_data(data, label, test_size=0.3)

train_images = []
train_labels = []
test_images = []
test_labels = []

for i in range(10):
    train_image = []
    train_label = []
    with open('dataset_2/train'+str(i)+'.txt') as f:
        for line in f:
            line = line.rstrip()
            l = line.split()
            img = cv2.imread(image_dir+'/'+l[0])
#             print(l[0])
            img = cv2.resize(img, (28, 28))

            train_image.append(img.flatten().astype(np.float32)/255.0)
            tmp = np.zeros(NUM_CLASSES)
            tmp[int(l[1])] = 1
            train_label.append(tmp)
        train_images.append(np.asarray(train_image))
        train_labels.append(np.asarray(train_label))

    test_image = []
    test_label = []
    with open('dataset_2/test'+str(i)+'.txt') as f:
        for line in f:
            line = line.rstrip()
            l = line.split()
            img = cv2.imread(image_dir+'/'+l[0])
            img = cv2.resize(img, (28, 28))

            test_image.append(img.flatten().astype(np.float32)/255.0)
            tmp = np.zeros(NUM_CLASSES)
            tmp[int(l[1])] = 1
            test_label.append(tmp)
        test_images.append(np.asarray(test_image))
        test_labels.append(np.asarray(test_label))

In [59]:
index = '2'
np.save('./dataset_'+index+'/data/train_images', train_images)
np.save('./dataset_'+index+'/data/train_labels', train_labels)
np.save('./dataset_'+index+'/data/test_images', test_images)
np.save('./dataset_'+index+'/data/test_labels', test_labels)

In [60]:
for train_image, train_label, test_image, test_label in zip(train_images, train_labels, test_images, test_labels):
    with tf.Graph().as_default():
        images_placeholder = tf.placeholder("float", shape=(None, IMAGE_PIXELS))
        labels_placeholder = tf.placeholder("float", shape=(None, NUM_CLASSES))

        #dropout率
        keep_prob = tf.placeholder("float")
        #モデル
        logits = inference(images_placeholder, keep_prob)
        #損失計算
        loss_value = loss(logits, labels_placeholder)
        #学習
        train_op = training(loss_value, learning_rate)
        #精度計算
        acc = accuracy(logits, labels_placeholder)
        #保存準備
        saver = tf.train.Saver()
        #セッションの作成
        sess = tf.Session()
        #変数の初期化
        sess.run(tf.global_variables_initializer())
        #Tensorboard表示用の設定
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(train_dir, sess.graph)

        for step in range(max_steps):
            for i in range(int(len(train_image)/batch_size)):
                batch = batch_size * i
                sess.run(train_op, feed_dict={
                    images_placeholder: train_image[batch: batch+batch_size],
                    labels_placeholder: train_label[batch: batch+batch_size],
                    keep_prob: 0.5
                })

            train_accuracy = sess.run(acc, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0
            })
            print("step %d, training accuracy %g"%(step, train_accuracy))

            summary_str = sess.run(summary_op, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0
            })
            summary_writer.add_summary(summary_str, step)

    print("test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0
    }))

    save_path = saver.save(sess, os.getcwd() + "¥¥model.ckpt")

step 0, training accuracy 0.539641
step 1, training accuracy 0.539641
step 2, training accuracy 0.539641
step 3, training accuracy 0.539641
step 4, training accuracy 0.540698
step 5, training accuracy 0.544926
step 6, training accuracy 0.553383
step 7, training accuracy 0.557611
step 8, training accuracy 0.564482
step 9, training accuracy 0.570825
step 10, training accuracy 0.578224
step 11, training accuracy 0.59408
step 12, training accuracy 0.600423
step 13, training accuracy 0.613636
step 14, training accuracy 0.61945
step 15, training accuracy 0.622622
step 16, training accuracy 0.642706
step 17, training accuracy 0.642178
step 18, training accuracy 0.656448
step 19, training accuracy 0.656977
step 20, training accuracy 0.661734
step 21, training accuracy 0.661205
step 22, training accuracy 0.678118
step 23, training accuracy 0.683404
step 24, training accuracy 0.67389
step 25, training accuracy 0.705603
step 26, training accuracy 0.682347
step 27, training accuracy 0.697674
step 

KeyboardInterrupt: 

In [77]:
    print("test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_images[9],
        labels_placeholder: test_labels[9],
        keep_prob: 1.0
    }))

test accuracy 0.784689
